## Change 2

1. Can you clarify what the model used previously when AmericanObsHoursPerDay ≠ 0?
2. When AmericanObsHoursPerDay = 0, how does the PDE solver handle time discretization? Could using actual time increase numerical instability in the PDE solver (e.g., irregular grid spacing)?
3. Does this change apply only to American (continuously monitored) barriers, or also to European-style (discretely monitored) barriers?
4. Does the actual observation hour setting now align with market trading hours across exchanges? How does the model handle different markets (e.g., US vs. Asia)?
5. What happens if a product is set with AmericanObsHoursPerDay = 0, but its calendar time to maturity is less than 1 day? Is that handled properly?
6. Once americanObsDt is computed, can you explain how it is fed into the PDE grid or ctsBarrierHitPct calculation? Is it treated as the total monitoring time?
7. Previously, was a fixed dt used for every barrier monitoring interval? Does the new implementation allow varying dt that reflects actual elapsed time (e.g., 3 days vs. 1 day)?
8. Have you compared the new results against analytical barrier models or market prices to confirm that using actual time improves accuracy? If so, please provide us the test. 

## Change 4

1. Can you confirm the theoretical basis for using the formula:
=POWER(1 + linear yield × term, 1 / term) - 1
as the flat equivalent compounding yield?

📌 Purpose: Ensure MD can justify the transformation from linear to compound terms.

2. Does pricing of embedded Bermudan swaption correctly handle accreting notional?
3. Is this approach valid only for linearly accreting notional with fixed yield?
How does the model behave if:

Notional grows non-linearly?

Accretion changes mid-life?

Call dates are not annual?

4. What threshold is used to judge “materiality” in the impact test? For instance, is a PV difference of $251k on $500M trade considered material?

In the upcoming release, MD will update the model so that when Product.AmericanObsHoursPerDay is set to zero, the PDE pricing method will use the actual time interval to compute the probability of crossing the barrier. For American (continuously monitored) barriers, this parameter determines the number of hours per day during which the barrier is observed (e.g., 7.5 hours per day).

To verify the impact of the updated parameter, MD used a sample trade in the attached spreadsheet DoubleBarrierVetting.xlsm to demonstrate how the model behaves under different observation settings. Three configurations were tested—24 hours, 24 hours including weekends, and the new "Actual" setting—each producing different barrier observation intervals.

For each configuration, the corresponding time step used by the PDE engine is shown via debugging screenshots. The observation window from May 30, 2025, to June 2, 2025, spans a weekend. Under the 24-hour setting, the model computes a time step of approximately 1/365 (~0.00274). With the 24-hour plus weekend setting, the time step increases to around 1.4/365 (~0.00384). Under the new "Actual" setting, the full 3-day interval is reflected, resulting in a time step of approximately 3/365 (~0.00822).

This behavior confirms that the model correctly captures the weekend effect under the new configuration and validates that the implementation aligns with expectations.








